# AUTOMATIC1111's Stable Diffusion WebUI

Adapted from: https://colab.research.google.com/drive/1Iy-xW9t1-OQWhb0hNxueGij8phCyluOh

**Updating the Notebook**

In the Tools section at the bottom of the notebook there is a block to download the latest version from Github.

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)
- [Stable Diffusion Models](https://cyberes.github.io/stable-diffusion-models)
- [Textual Inversion Models](https://cyberes.github.io/stable-diffusion-textual-inversion-models/)
- [Have I Been Trained?](https://haveibeentrained.com/)

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account.

`/notebooks/` is storage for this notebook only.

<mark style="background-color:lime">`/tmp/` is not a persistent directory, meaning your files there will be deleted when the machine turns off.</mark>

<br>

If you are having storage issues, set `repo_storage_dir` to `/tmp/stable-diffusion`.

<br>

<mark>You must uncomment the correct section and run the block below or else the notebook won't work!</mark>

In [ ]:
# Free tier
# free_tier = True                                     # Enables the creation of symlinks back to /notebooks/
# model_storage_dir = '/tmp/stable-diffusion/models'   # Where the models will be downloaded to
# repo_storage_dir = '/notebooks'                      # Where the repository will be downloaded to

# Paid Tier
# free_tier = False
# model_storage_dir = '/storage/models'
# repo_storage_dir = '/notebooks'

# Don't put a trailing slash on directory paths.
# To reset your storage directory, rerun this cell.

# ===============================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store free_tier model_storage_dir repo_storage_dir

### Clone the central repository

In [ ]:
import os
%store -r free_tier model_storage_dir repo_storage_dir
%cd /notebooks/

def delete_broken_symlinks(path):
    # make sure to pass this function a path without a trailing slash
    for file in os.listdir(path):
        if os.path.islink(f'{path}/{file}') and not os.path.exists(os.readlink(f'{path}/{file}')):
            print(f'Symlink broken, removing: {file}')
            os.unlink(f'{path}/{file}')


if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui'):
    if free_tier:
        delete_broken_symlinks('/notebooks/') # remove broken symlinks since it might have been installed in a non-persistent directory
        if not os.path.exists(repo_storage_dir):
            !mkdir -p "{repo_storage_dir}"
            !ln -s "{repo_storage_dir}" /notebooks/
    %cd "{repo_storage_dir}"
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else: # update repo if already exists
    print('stable-diffusion-webui already downloaded, updating...')
    %cd "{repo_storage_dir}/stable-diffusion-webui"
    !git pull

### Install requirements and download repositories

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
%cd "{repo_storage_dir}/stable-diffusion-webui"

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
import launch

# latent-diffusion is a requirement but launch.py isn't downloading it so we'll do it manually.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion'):
    !git clone https://github.com/crowsonkb/k-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/k-diffusion"
    !git clone https://github.com/Hafiidz/latent-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion"
    # I don't think it's necessary to do this:
    # %mkdir "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion/experiments/"
    # %mkdir "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion/experiments/pretrained_models"
    # !wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/lfs/latent-diffusion/project.yaml -O "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion/experiments/pretrained_models/project.yaml"
    # !wget https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/lfs/latent-diffusion/model.ckpt?raw=true -O "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion/experiments/pretrained_models/model.ckpt"

# Download the GFPGAN face restorer.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -O "{repo_storage_dir}/stable-diffusion-webui/GFPGANv1.3.pth"
else:
    print('GFPGANv1.3.pth already downloaded')
    
# Make sure your models storage directory exists
!mkdir -p "{model_storage_dir}"

# Link the output folder to /notebooks/outputs
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/outputs"
!ln -s "{repo_storage_dir}/stable-diffusion-webui/outputs" /notebooks/outputs

### Download the Model

I've provided a few different ways of aquiring the models. Try the torrent option first. You don't need to repeat this step if you've already downloaded the models.

There are a few additional models available here: https://cyberes.github.io/stable-diffusion-models

If you're interested in textual inversion, here's the database: https://cyberes.github.io/stable-diffusion-textual-inversion-models

**Filesize and Storage Disclaimer**

Paperspace free tier has only 5GB of storage space. If you're having storage issues, here's a few suggestions.
1. Download everything to `/tmp/`
2. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
3. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
4. Use my referral code `KQLRH37` You'll get \$10 credit that you should be able to put towards the storage overage charges. Redeem the code at the bottom of the Billing page.

If you're on free tier, only download one model.

**Torrent Instructions**

Aria2 may show some errors/warnings while downloading. Those are fine, when it eventually says "Download Complete" that means everything worked as it should.

#### Standard Model

**Torrent**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337/announce"

**Web Download**

Voldy provided an alternative download if you don't want to use HuggingFace.

[https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view](https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view)

Download it to your computer then upload it to your model storage directory (make sure it's named `sd-v1-4.ckpt`).

HuggingFace is much faster and reliable but you need to get access to the repo and provide your user token.

In [ ]:
user_token = "<put your user token here>"

# ===============================================================================================
%store -r free_tier model_storage_dir repo_storage_dir
!wget --header="Authorization: Bearer {user_token}" https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O "{model_storage_dir}/sd-v1-4.ckpt"

#### Waifu Diffusion

The original Stable Diffusion anime finetune.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://tracker.opentrackr.org:1337/announce"

#### trinart_stable_diffusion_v2

Another anime finetune. Designed to nudge SD to an anime/manga style. Seems to be more "stylized" and "artistic" than Waifu Diffusion, if that makes any sense.

The 60,000 steps version is the original, the 115,000 and 95,000 versions is the 60,000 with additional training. Use the 60,000 step version if the style nudging is too much.

[See the comparison here.](https://cyberes.github.io/stable-diffusion-models/#model-comparison)

**60000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt -O "{model_storage_dir}/trinart2_step60000.ckpt"

**95000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt -O "{model_storage_dir}/trinart2_step95000.ckpt"

**115000**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!wget https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt -O "{model_storage_dir}/trinart2_step115000.ckpt"

### Clean up and restart the kernel

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir

# Get some storage back
!pip cache purge
!cd "{model_storage_dir}" && rm *.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

### Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
import os

# Check for broken symlinks and remove them
deleted = False
for file in os.listdir(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/'):
    path = f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{file}'
    if os.path.islink(path) and not os.path.exists(os.readlink(path)):
        print(f'Symlink broken, removing: {file}')
        os.unlink(path)
        deleted = True
if deleted:
    print('')

# Make symlinks for new files
for file in os.listdir(model_storage_dir):
    if file.endswith("ckpt"):
        path = f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{file}'
        if not os.path.exists(path):
            print(f'New model: {file}')
            !ln -s "{model_storage_dir}/$file" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/$file"
        !ls -la --block-size=GB "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/$file"
        print('')

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

If you have a lot of VRAM and aren't generating large images you can add the flag `--disable-opt-split-attention` to disable VRAM optimizations for a speed boost.

**Troubleshooting**

- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files or you can reset your storage with the _Reset Storage_ block in the Tools section below.
- If you're having issues with your results not loading, try running an ngrok proxy. More details in the Tools section below.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
%cd "{repo_storage_dir}/stable-diffusion-webui"
!python webui.py --gradio-debug --share # --gradio-auth me:password1234

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/stable-diffusion-webui/`.

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
!apt update
!apt install -y p7zip-full
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd /notebooks/
!mkdir "{datetime_str}"
!mv "{repo_storage_dir}/stable-diffusion-webui/log" "/notebooks/{datetime_str}"
!cd "{repo_storage_dir}/stable-diffusion-webui/outputs/" && mv * "/notebooks/{datetime_str}"
!TEMP="/notebooks/{datetime_str}" # find command has issues with ipynb variables??
# !find $TEMP -name .ipynb_checkpoints -exec rm -rf "{}" +
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "/notebooks/{datetime_str}/"

### Delete old output folder

This block will delete the folder you just compressed.

In [ ]:
!rm -rf "/notebooks/{datetime_str}/"
!echo Deleted /notebooks/{datetime_str}/

# Tools

### Show graphics card info

In [ ]:
!nvidia-smi

### Download the latest version of this notebook from Github

Run this and refresh the page (press F5). Don't save anything or you will overwrite the downloaded file.

In [ ]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup # save your old notebook to a backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb

### Reset Repository

Sometimes AUTOMATIC1111 breaks something. Go to https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master and choose a commit to revert to.

**This shouldn't delete your outputs or any changes you've made to files, but I'd back up anything important just to be safe.**

In [ ]:
%store -r free_tier model_storage_dir repo_storage_dir
%cd "{repo_storage_dir}/stable-diffusion-webui"
!git reset --hard <commit>

### Reset storage

This will delete ALL your files in `/notebooks/`, `/storage/`, `model_storage_dir`, and `repo_storage_dir`. Use if you're having issues with zero storage space and you don't want to delete your notebook.

In [ ]:
# Uncomment the lines below to run this block. You can highlight the lines and do ctrl + /
# %store -r free_tier model_storage_dir repo_storage_dir
# !rm -rf /storage/*
# !mv /notebooks/*.ipynb / # move the notebook out of the directory before we nuke it
# !rm -rf /notebooks/*
# !mv /*.ipynb /notebooks/ # move it back
# !rm -rf {model_storage_dir}
# !rm -rf {repo_storage_dir}